In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [25]:
# Ebola 데이터 읽기
ebola = pd.read_csv('data/ebola_2014-2016.csv')

In [27]:
# Ebola의 Date 컬럼을 datetime 타입으로 변경
ebola.Date = pd.to_datetime(ebola['Date'], format='%Y-%m-%d')

In [35]:
# 데이터를 2014, 2015, 2016으로 나누기
mask_2014 = (ebola['Date'] >= '2014-01-01') & (ebola['Date'] <= '2014-12-31')
mask_2015 = (ebola['Date'] >= '2015-01-01') & (ebola['Date'] <= '2015-12-31')
mask_2016 = (ebola['Date'] >= '2016-01-01') & (ebola['Date'] <= '2016-12-31')

In [36]:
ebola2014 = ebola.loc[mask_2014]
ebola2015 = ebola.loc[mask_2015]
ebola2016 = ebola.loc[mask_2016]

In [55]:
mask_2015_first = (ebola2015['Date'] >= '2015-01-01') & (ebola2015['Date'] <= '2015-06-30')
mask_2015_second = (ebola2015['Date'] >= '2015-07-01') & (ebola2015['Date'] <= '2015-12-31')

mask_2016_first = (ebola2016['Date'] >= '2016-01-01') & (ebola2016['Date'] <= '2016-06-30')
mask_2016_second = (ebola2016['Date'] >= '2016-07-01') & (ebola2016['Date'] <= '2016-12-31')

In [56]:
ebola2015_first = ebola2015.loc[mask_2015_first]
ebola2015_second = ebola2015.loc[mask_2015_second]

ebola2016_first = ebola2016.loc[mask_2016_first]
ebola2016_second = ebola2016.loc[mask_2016_second]

In [53]:
# ebola2014
# ebola2015_first, ebola2015_second
# ebola2016_first, ebola2016_second



,Country,Date,Cases,Deaths
0,Guinea,2014-08-29,648.0,430.0
1,Nigeria,2014-08-29,19.0,7.0
2,Sierra Leone,2014-08-29,1026.0,422.0
3,Liberia,2014-08-29,1378.0,694.0
4,Sierra Leone,2014-09-05,1261.0,491.0
...,...,...,...,...
260,Sierra Leone,2014-12-31,9446.0,2758.0
261,Mali,2014-12-31,8.0,6.0
262,Nigeria,2014-12-31,20.0,8.0
263,United States of America,2014-12-31,4.0,1.0
